In [1]:
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")
from plotly import io
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
file = pd.read_csv("diabetes.csv")
file.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [3]:
encoder = MinMaxScaler()
lable = file.columns

In [4]:
en_file = encoder.fit_transform(file.values)

In [5]:
en = pd.DataFrame(en_file, columns=lable)
en.head(3)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.0,0.500745,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.0,0.396423,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.0,0.347243,0.253629,0.183333,1.0


In [6]:
feature = en.values[:,:-1]
label = en.values[:,-1]

In [7]:
import torch
import torchvision
import torchvision.transforms as transforms

In [17]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc4 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc4(x)
        return x


net = Net()

In [18]:
inputs = torch.Tensor(feature)
labels = torch.Tensor(label).long()

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [20]:
for epoch in range(int(1e4)):
    running_loss = 0.0
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    print('loss: %.3f' %( running_loss)) if epoch%1000==0 else ""

print('Finished Training')6000

loss: 0.895
loss: 0.386
loss: 0.266
loss: 0.231
loss: 0.219
loss: 0.212
loss: 0.206
loss: 0.202
loss: 0.198
loss: 0.195
Finished Training


In [21]:
correct = 0
total = 0
with torch.no_grad():
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network %d %%' % (100 * correct / total))

Accuracy of the network 92 %
